In [6]:
import pandas as pd
import numpy as np
import pandas as pd
import re
from collections import Counter
pd.set_option('display.max_columns', 500)

data = pd.read_excel('anonymized_Learner_Corpus.xlsx')

In [7]:
data

,LEARNER ID,Major,Age,Term 1 Instructor,Term 2 Instructor,Term 3 Instructor,Term 4 Instructor,Mother's native language,Languages spoken at home,Primary school language,Secondary school language,English Instruction,Stay in an English speaking country,Other foreign languages,Reference tools use,Reference tools,Which bilingual dictionary?,Which monolingual dictionary?,grammar references,Practice Paragraph,Task 1.1.1,Task 1.1.2,Task 1.2.1,Task 1.2.2,Task 2.1.1,Task 2.1.2,Task 2.2.1,Task 2.2.2,Exam Term 2,Essay 3.1.1,Essay 3.1.2,Essay 3.2.1,Essay 3.2.2,Exam Term 3,Task 4.1,Task 4.2,Task 4.3
0,10576,Nursing,18,Instr026,Instr055,Instr015,Instr019,Arabic,Arabic,Arabic,Arabic,NaN,NaN,NaN,Yes,Grammarly,NaN,NaN,NaN,One way to shop from home is to use a virtual ...,NaN,<title>Reasons for Buying an IPad:</title> \n\...,NaN,<title>Causes of Eating Disorders Among Young ...,There are many differences between WhatsApp ap...,There are many differences between the WhatsAp...,NaN,<title>Drawing</title> \n\n\nDrawing is a Visi...,NaN,NaN,<title>Is it Important to Study a Foreign Lang...,NaN,<title>Substance Abuse and Addiction in Teenag...,<title>Causes and Effects of Playing Video Gam...,<title>Stressors in University Life and Anxiet...,<title>A Critique of “Stressors in University...,<title>Final Project \n\nA Review of “Stressor...
1,10553,Nursing,18,Instr044,Instr033,Instr049,Instr050,Arabic,Arabic,Arabic,Arabic,9,NaN,NaN,Yes,Google,Oxford,Oxford,NaN,\n\n \n\n<title>In what ways did smartphones ...,NaN,<title>Reasons for having /Buying an iPad<titl...,NaN,<title>The effects of daily exercise<title>\n\...,<title>Twitter vs WhatsApp<title>\n\nMany peop...,<title>Twitter vs WhatsApp<title> \n\n \n\nMan...,"<title>Swimming<title> \n \n \n"" Swimming is o...","<title>Swimming<title> \n \n"" To move in the ...",<title>summer season Vs winter season</title>\...,<title>Is it important to study a foreign lang...,<title>Is It Important to Study a Foreign Lang...,<title>What are the causes and effects of subs...,<title>What are the causes and effects of subs...,For university students? What are the causes ...,<title>Summary of (Alcohol abuse in older adul...,<title>Critique</title> \n \nToday the world f...,<title>Final project</title>\n \n D...
2,10170,AMS,18,Instr031,Instr057,Instr041,Instr052,Arabic,Arabic,Arabic,Arabic,7-10 years,NaN,NaN,Yes,Dictionary,Oxford,Merriam Webster and Oxford,textbooks,<title>Advantages of Travelling</title> \n \n ...,NaN,<title>Advantages of Travelling</title>\n \n ...,NaN,<title>Causes of Eating Disorders Among Young ...,\n <title>Role of a father vs the role of a m...,\n<title>Role of a father vs the role of a mo...,<title>An Ideal friend</title> \n \n \nAn ide...,<title>An Ideal Friend</title> \n \n \nAn ide...,NaN,<title>Providing Textbooks for free</title> \...,<title>Providing Textbooks for Free</title> \...,<title>The causes and effect of divorce</title...,<title>The Causes and Effects of Divorce</titl...,<title>Causes and Effects of Lack of Sleep</ti...,<title>The effect of Emotional Freedom Techniq...,<title>The effect of Emotional Freedom Techniq...,<title>The effect of Emotional Freedom Techniq...
3,10599,Nursing,18,Instr006,Instr023,Instr023,Instr049,Arabic,Arabic,Arabic,Arabic and English,7-10 years,NaN,NaN,Yes,Dictionary,NaN,Oxford,NaN,<title>Types of part time jobs that are suitab...,NaN,<title>.Advantages of living in a dormitory</t...,NaN,<title>.Causes of eating disorders among young...,<title>Volunteering VS Part-time job</title> \...,<title>Volunteering VS Part-Time Job</title> \...,<title>Mental health</title> \n \n \n Many...,<title>Mental health</title>\n \n \n \t\n\n...,NaN,<title>Are textbooks essential for learning in...,<title>textbooks are not essential for learni...,<title>The Causes and Effects of Cyberbullying...,<title>The Causes and Effects of Cyberbullyi...,<title>Causes and The Effect of <original=Chos...,<title>Usage of Glucagon-Like peptide-1 for Ob...,<title>A Critique of “ Usage of G

In [5]:
!pip3 install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 250 kB 712 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Extracting mistakes from the data

In [10]:
import os

def mistakes_data_analysis(file, text_columns, output_folder = 'analysis', output_filename = 'mistakes.csv'):
    """"
    # TODO: write proper summary of how to use this function
    """
    data = pd.read_excel(file)

    all_mistakes_list = []

    for column in text_columns:
        if column in data.columns:
            for text_id, text in zip(data['Major'],  data[column].dropna()):
                if not pd.isnull(text):

                    corrected_text = re.sub(r"<original=([^>']+)>", r"<original='\1'>", text)

                    mistakes = re.findall(r"<original='([^']+)'>([^<]+)</original>", corrected_text)

                    all_mistakes_list.extend([{'Major': text_id, "Original":mistake[0], 'Corrected':mistake[1] } for mistake in mistakes])

    mistakes_data = pd.DataFrame(all_mistakes_list, columns=['Major', 'Original', 'Corrected'])

    os.makedirs(output_folder, exist_ok=True)

    output_path = os.path.join(output_folder, output_filename)
    mistakes_data.to_csv(output_path, index=False, header=True)

    return mistakes_data

text_columns = [
       'Practice Paragraph', 'Task 1.1.1', 'Task 1.1.2', 'Task 1.2.1',
       'Task 1.2.2', 'Task 2.1.1', 'Task 2.1.2', 'Task 2.2.1', 'Task 2.2.2',
       'Exam Term 2', 'Essay 3.1.1', 'Essay 3.1.2', 'Essay 3.2.1',
       'Essay 3.2.2', 'Exam Term 3', 'Task 4.1', 'Task 4.2', 'Task 4.3',
]


# mistakes extraction

mistakes_data_analysis('anonymized_Learner_Corpus.xlsx', text_columns)


,Major,Original,Corrected
0,Nursing,producis,products
1,Nursing,Times,teams
2,Nursing,Coll,call
3,Nursing,live,life
4,Nursing,sand,send
...,...,...,...
8433,Nursing,whit,with
8434,Med,enemia,anemia
8435,Med,preform,perform
8436,Med,standers,standards


# Calculate word count

In [11]:
def calculate_word_counts(file, text_columns, output_folder = 'analysis', output_filename = 'word_count.csv', include_reference = False):
    """
        Calculate word counts for specified text columns in an Excel file and save the results to a Excel file.

        Parameters:
            - file (str): Path to the Excel file containing the data.
            - text_columns (list): List of column names that contain text data for word count calculation.
            - output_folder (str): Directory where the output CSV file will be saved. Default is 'analysis'.
            - output_filename (str): Name of the output CSV file. Default is 'word_count.csv'.
            - include_reference (bool): Whether to include reference sections in the word count. Default is False.

        Returns:
            - pd.Dataframe -> with word count
    """
    data = pd.read_excel(file)

    def student_words_count(text):
        """
        #TODO: docstrings to explain what this function is doing
        """
        if pd.isnull(text):
            return 0
        cleaned_text = re.sub(r'<reference_list>.*?</reference_list>', '', text, flags=re.DOTALL) # ignores reference_list
        cleaned_text = re.sub(r"<[^>]+>|[^\w\s']", "", cleaned_text)
        cleaned_text = cleaned_text.replace('\n', ' ').strip()
        words = cleaned_text.split()
        return len(words)

    def word_count(text): # is couting words with reference list
        """
        #TODO: docstrings to explain what this function is doing
        """
        if pd.isnull(text):
            return 0
        cleaned_text = re.sub(r"<[^>]+>|[^\w\s']", "", text) # removes HTML tags and non-word characters except spaces and apostrophes
        cleaned_text = cleaned_text.replace('\n', ' ').strip() # replacing newline (\n) characters with spaces
        words = cleaned_text.split()
        return len(words)

    if include_reference == False:
        for column in text_columns:
            if column in data.columns:
                data[column + '_word_count'] = data[column].apply(student_words_count)

        data['Total_Word_Count'] = data[[col + '_word_count' for col in text_columns]].sum(axis = 1)
    else:
        for column in text_columns:
            if column in data.columns:
                data[column + '_word_count'] = data[column].apply(word_count)

        data['Total_Word_Count'] = data[[col + '_word_count' for col in text_columns]].sum(axis = 1)

    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_filename)
    data.to_csv(output_path, index=False, header=True)

    return data

# applying word count
calculate_word_counts('anonymized_Learner_Corpus.xlsx', text_columns)


,LEARNER ID,Major,Age,Term 1 Instructor,Term 2 Instructor,Term 3 Instructor,Term 4 Instructor,Mother's native language,Languages spoken at home,Primary school language,Secondary school language,English Instruction,Stay in an English speaking country,Other foreign languages,Reference tools use,Reference tools,Which bilingual dictionary?,Which monolingual dictionary?,grammar references,Practice Paragraph,Task 1.1.1,Task 1.1.2,Task 1.2.1,Task 1.2.2,Task 2.1.1,Task 2.1.2,Task 2.2.1,Task 2.2.2,Exam Term 2,Essay 3.1.1,Essay 3.1.2,Essay 3.2.1,Essay 3.2.2,Exam Term 3,Task 4.1,Task 4.2,Task 4.3,Practice Paragraph_word_count,Task 1.1.1_word_count,Task 1.1.2_word_count,Task 1.2.1_word_count,Task 1.2.2_word_count,Task 2.1.1_word_count,Task 2.1.2_word_count,Task 2.2.1_word_count,Task 2.2.2_word_count,Exam Term 2_word_count,Essay 3.1.1_word_count,Essay 3.1.2_word_count,Essay 3.2.1_word_count,Essay 3.2.2_word_count,Exam Term 3_word_count,Task 4.1_word_count,Task 4.2_word_count,Task 4.3_word_count,Total_Word_Count
0,10576,Nursing,18,Instr026,Instr055,Instr015,Instr019,Arabic,Arabic,Arabic,Arabic,NaN,NaN,NaN,Yes,Grammarly,NaN,NaN,NaN,One way to shop from home is to use a virtual ...,NaN,<title>Reasons for Buying an IPad:</title> \n\...,NaN,<title>Causes of Eating Disorders Among Young ...,There are many differences between WhatsApp ap...,There are many differences between the WhatsAp...,NaN,<title>Drawing</title> \n\n\nDrawing is a Visi...,NaN,NaN,<title>Is it Important to Study a Foreign Lang...,NaN,<title>Substance Abuse and Addiction in Teenag...,<title>Causes and Effects of Playing Video Gam...,<title>Stressors in University Life and Anxiet...,<title>A Critique of “Stressors in University...,<title>Final Project \n\nA Review of “Stressor...,199,0,137,0,115,254,256,0,285,0,0,458,0,477,326,364,459,826,4156
1,10553,Nursing,18,Instr044,Instr033,Instr049,Instr050,Arabic,Arabic,Arabic,Arabic,9,NaN,NaN,Yes,Google,Oxford,Oxford,NaN,\n\n \n\n<title>In what ways did smartphones ...,NaN,<title>Reasons for having /Buying an iPad<titl...,NaN,<title>The effects of daily exercise<title>\n\...,<title>Twitter vs WhatsApp<title>\n\nMany peop...,<title>Twitter vs WhatsApp<title> \n\n \n\nMan...,"<title>Swimming<title> \n \n \n"" Swimming is o...","<title>Swimming<title> \n \n"" To move in the ...",<title>summer season Vs winter season</title>\...,<title>Is it important to study a foreign lang...,<title>Is It Important to Study a Foreign Lang...,<title>What are the causes and effects of subs...,<title>What are the causes and effects of subs...,For university students? What are the causes ...,<title>Summary of (Alcohol abuse in older adul...,<title>Critique</title> \n \nToday the world f...,<title>Final project</title>\n \n D...,148,0,120,0,90,197,183,159,161,276,328,325,447,442,253,307,397,668,4501
2,10170,AMS,18,Instr031,Instr057,Instr041,Instr052,Arabic,Arabic,Arabic,Arabic,7-10 years,NaN,NaN,Yes,Dictionary,Oxford,Merriam Webster and Oxford,textbooks,<title>Advantages of Travelling</title> \n \n ...,NaN,<title>Advantages of Travelling</title>\n \n ...,NaN,<title>Causes of Eating Disorders Among Young ...,\n <title>Role of a father vs the role of a m...,\n<title>Role of a father vs the role of a mo...,<title>An Ideal friend</title> \n \n \nAn ide...,<title>An Ideal Friend</title> \n \n \nAn ide...,NaN,<title>Providing Textbooks for free</title> \...,<title>Providing Textbooks for Free</title> \...,<title>The causes and effect of divorce</title...,<title>The Causes and Effects of Divorce</titl...,<title>Causes and Effects of Lack of Sleep</ti...,<title>The effect of Emotional Freedom Techniq...,<title>The effect of Emotional Freedom Techniq...,<title>The effect of Emotional Freedom Techniq...,136,0,167,0,279,239,228,250,296,0,478,464,510,481,324,315,567,797,5531
3,10599,Nursing,18,Instr006,Instr023,Instr023,Instr049,Arabic,Arabic,Arabic,Arabic and English,7-10 years,NaN,NaN,Yes,Dictionary,NaN,Oxford,NaN,<title>Types of part time jobs that are suitab...,NaN,<t

# Word frequency

In [12]:
def clean_and_analyze_text(data, text_columns, output_folder='analysis', output_filename='word_frequency.csv'):
    """
    Cleans the text data in specific columns of a given DataFrame by removing specific HTML-like tags
    such as <reference_list>, <references_list>, and <in_text_reference>, along with
    their enclosed content. It also removes any instances of 'reference_list' or
    'references_list' as extracted words, and calculates word frequency.

    Parameters:
        data (pd.DataFrame): A pandas DataFrame containing text data.
        text_columns (list): List of columns containing text to be processed.
        output_folder (str): The folder where the output file will be saved (default is 'output').
        output_filename (str): The name of the output CSV file (default is 'word_frequency.csv').

    Returns:
        pd.DataFrame: A cleaned DataFrame where word frequency is calculated and saved.
    """


    combined_words = ' '.join(data[text_columns].astype(str).fillna('').values.flatten())


    cleaned_content = re.sub(r'<references?_list>[\s\S]*?</references?_list>', '', combined_words, flags=re.DOTALL)
    cleaned_content = re.sub(r'<in_text_reference>[\s\S]*?</in_text_reference>', '', cleaned_content, flags=re.DOTALL)
    corrected_text = re.sub(r"<original='.*?'>(.*?)</original>", r'\1', cleaned_content)

    cleaned_content_new = corrected_text.lower()

    words = re.findall(r"\b\w+(?:[-']\w+)*\b", cleaned_content_new)
    words = [word for word in words if word not in ['reference_list', 'references_list']]

    word_count = Counter(words)
    word_count_df = pd.DataFrame(word_count.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False)

    os.makedirs(output_folder, exist_ok = True)

    output_path = os.path.join(output_folder, output_filename)

    word_count_df.to_csv(output_path, index=False, header=True)

    return word_count_df

text_columns = [
       'Practice Paragraph', 'Task 1.1.1', 'Task 1.1.2', 'Task 1.2.1',
       'Task 1.2.2', 'Task 2.1.1', 'Task 2.1.2', 'Task 2.2.1', 'Task 2.2.2',
       'Exam Term 2', 'Essay 3.1.1', 'Essay 3.1.2', 'Essay 3.2.1',
       'Essay 3.2.2', 'Exam Term 3', 'Task 4.1', 'Task 4.2', 'Task 4.3',
]


clean_and_analyze_text(data, text_columns)

,Word,Frequency
33,the,40627
25,and,31310
32,of,26205
2,to,23638
8,a,19900
...,...,...
8864,lowestwith,1
14663,helith,1
14662,phecsil,1
14661,illusatrte,1


In [16]:
import re
def word_frequncy(data, text_columns, output_folder = 'analysis', output_filename = 'word_frequency.csv'):
    """
    # TODO: doc
    """
    data = pd.read_excel(data)

    combined_test_v2 = ''

    for col in text_columns:
        text_data = data[col].astype(str).str.cat(sep = ' ')
        text_data = re.sub(r'<reference_list>.*?</reference_list>', '', text_data, flags=re.DOTALL)
        combined_test_v2 = ' ' + text_data

    word_2 = re.findall(r'\b\w+\b', combined_test_v2.lower())
    word_frequncy = Counter(word_2)

    word_frequncy_df = pd.DataFrame(word_frequncy.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False)

    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_filename)
    word_frequncy_df.to_csv(output_path, index=False, header=True)

    return word_frequncy_df

In [18]:
len(text_columns)

18

In [17]:
word_frequncy('data/anonymized_Learner_Corpus.xlsx', text_columns)

,Word,Frequency
25,the,8489
5,of,4778
10,and,4516
58,in_text_reference,3432
32,to,3030
...,...,...
4821,fashion,1
4822,plumpness,1
4823,maintenance,1
4824,conflict,1
